In [ ]:
import numpy as np
import nibabel as nib
import os
from matplotlib import pyplot as plt
import pathlib
import vtk
import h5py
from statsmodels.stats.multitest import multipletests
from scipy.stats import norm
import pandas as pd
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, precision_score, recall_score, f1_score,confusion_matrix


In [ ]:

results = pd.DataFrame()
stride = 1
threshold = 0.5
for m in range(4,9):
    fold = foldraw[foldraw['age']//stride==m]
    # fold = fold[fold['fold']==0]
    print(f'Age:[{stride*m},{stride*(m+1)}),Number: {len(fold)}, Positive: {(fold["labels"]==1).sum()}')
    labels = fold['labels'].values
    preds = fold['preds'].values
    fpr,tpr,threshold = roc_curve(labels,preds)
    plt.figure()
    plt.plot(fpr,tpr)
    plt.plot([0,1],[0,1],'--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve of Age:[{},{})'.format(stride*m,stride*(m+1)))
    plt.savefig(f'roc_{m}.png',dpi=300)
    threshold = 0.5
#     print('Threshold:',threshold)
    tn, fp, fn, tp = confusion_matrix(labels,preds>threshold).ravel()
    print('Accuracy: {:.3f}'.format((tn+tp)/(tn+fp+fn+tp)))
    print('Specificity: {:.3f}, Sensitivity: {:.3f}'.format(tn/(tn+fp),tp/(tp+fn)))
    print('AUC: {:.3f}'.format(roc_auc_score(labels,preds)))
#     for threshold in np.linspace(0,1,40):
        

#     predsi = preds>0.5
#     mixed = [((labels==0)&(predsi==labels)).sum(),
#             ((labels==0)&(predsi!=labels)).sum(),
#             ((labels==1)&(predsi==labels)).sum(),
#             ((labels==1)&(predsi!=labels)).sum()]
#     prec = mixed[0]/(mixed[0]+mixed[1]+1e-6)
#     recall = mixed[0]/(mixed[0]+mixed[3]+1e-6)
#     f1 = 2*prec*recall/(prec+recall+1e-6)
#     print(f'Precision: {prec:.2f}, Recall: {recall:.2f}, AUC: {roc_auc_score(labels,preds):.2f}, Accuracy: {(mixed[0]+mixed[2])/sum(mixed):.2f}')


In [ ]:
(pd.read_csv('./fold.csv')['DX_GROUP']==1).sum()

In [ ]:
data = nib.load('/data04/301_postprocess/301_embed/tractoembedding/sub-14117/tractoembedding/da-full/sub-14117-trace2_CLR_sz320.nii.gz').get_fdata()

In [ ]:
demo = pd.read_csv('sub_info.csv')

demo_raw = pd.read_csv('new_400_301_fine_useful.csv')

# add age from demo to demo raw respect to SUB_ID

demo_raw['age'] = np.nan
demo_raw['sex'] = np.nan
for i in range(len(demo_raw)):
    demo_raw.loc[i, 'age'] = demo[demo['SUB_ID'] == demo_raw.loc[i, 'SUB_ID']]['age'].values[0]
    demo_raw.loc[i,'sex'] = demo[demo['SUB_ID'] == demo_raw.loc[i, 'SUB_ID']]['sex'].values[0]

In [ ]:
demo_raw1 = demo_raw[demo_raw['age']<11]
demo_raw1['age_group'] = np.round(demo_raw1['age']) // 2 
demo_raw1.dropna(axis=0,inplace=True)

demo_raw1['final_group'] = demo_raw1['age_group']*10 + demo_raw1['sex']
for i in np.unique(demo_raw1['final_group']):
    # print(i, len(demo_raw1[demo_raw1['final_group'] == i]))
    demo_raw1[demo_raw1['final_group']==i]
    sub_id = demo_raw1.loc[demo_raw1['final_group']==i,'SUB_ID'].to_numpy()

    np.random.shuffle(sub_id)
    
    for i in range(5):
        group_list = sub_id[i*len(sub_id)//5:(i+1)*len(sub_id)//5]
        print(i, group_list)
        demo_raw1.loc[demo_raw1['SUB_ID'].isin(group_list),'fold']=i
        # demo_raw1.loc[demo_raw1['SUB_ID'].isin(group_list),'fold'] = i

In [ ]:
import seaborn as sns
for i in range(5):
    i = float(i)
    demo_raw1.loc[demo_raw1['fold'] == i,'fold_new'] = 1
    demo_raw1.loc[demo_raw1['fold'] != i,'fold_new'] = 0
        # demo_raw1.loc[data_5_fold[i],'fold'] = i
    # for i, fold in enumerate(data_5_fold):
    #     if i == 2:
    #         demo_raw1.loc[fold,'fold'] = 1
    #     else:
    #         demo_raw1.loc[fold,'fold'] = 0
    #     print(demo_raw1.loc[fold, 'age'].mean())
    #     print(demo_raw1.loc[fold, 'age'].std())
    # sns.histplot(demo_raw['age'], bins=20)
    # demo_raw1 = demo_raw[demo_raw['age']<14]
    fig = sns.displot(x='age', data=demo_raw1, kind='hist',col='fold_new', bins=40,hue='sex')
    plt.savefig(f'dis_fig/age_distribution{i :.0f}.png', dpi=300)


In [ ]:
def if_pos(pos,name):
    if pos=='left':
        if pos in name:
            return True
        else:
            return False
    if pos=='right':
        if pos in name:
            return True
        else:
            return False
    if pos=='commisural':
        if 'left' not in name and 'right' not in name:
            return True
        
    

In [ ]:
# load vtk data
filename='/data01/zixi/tractoembedding_1000/749361/tracts/749361.vtp'
reader = vtk.vtkXMLPolyDataReader()
reader.SetFileName(filename)
reader.Update()
outpd = reader.GetOutput()
# print(embed.shape)

In [ ]:
D = 320
mask_d = 320
pos = 'left'
from itertools import product
for mask_d, pos in product([320],['left','right','commisural']):
    embed = np.load('/data01/zixi/tractoembedding_1000/749361/embed_feat.npy')
    mask = np.load(f'attns/precision_sz{mask_d}_mode_{pos}_total.npy').squeeze()
    f = h5py.File('/data01/zixi/tractoembedding_1000/749361/tracts/749361.h5', "r")
    tract_list = f['tract_list'][:]
    tract_name = f['tract_name'][:]
    tracts_id = np.concatenate([np.where(tract_list==i)[0] for i,name in enumerate(tract_name) if if_pos(pos,str(name))])
    print(tracts_id)
    for name in np.unique(tract_name[tract_list[tracts_id].astype(int)]):
        write_vtp(outpd,tracts_id,str(name))

#     embed = embed[tracts_id,:]
#     embedding_D = embed*50
#     dx = embedding_D[:, 0].round() + D / 2 - D / 10
#     dy = embedding_D[:, 1].round() + D / 2 - D / 10
#     plt.figure(figsize=(5,5))
#     plt.scatter(dx, dy)
#     plt.xlim(0, D)
#     plt.ylim(D, 0)
#     mean,std = mask.mean(),mask.std()
#     zscore = (mask-mean)/std
#     pvals = 2*(1-norm.cdf(np.abs(zscore)))
#     pvals_flat = pvals.flatten()
#     sign_mask, pvals_corrected,_,_ = multipletests(pvals_flat, alpha=0.01, method='bonferroni')
#     pvals_corrected = pvals_corrected.reshape(pvals.shape)
#     sign_mask = sign_mask.reshape(pvals.shape)
#     mask  = sign_mask
#     x,y = np.where(mask)
#     index = np.where(np.bitwise_and(dx[...,None]==x[None,...],dy[...,None]==y[None,...,]).sum(1).astype(bool))
#     embed_masked = embed[index,:]
#     plt.scatter(dx[index], dy[index])
#     plt.title(f'{pos}, Resolusion:{mask_d}')
#     # plt.figure(figsize=(5,5))
#     plt.imshow(mask.T,alpha=1,cmap='gray')
#     plt.xticks([])
#     plt.yticks([])
#     plt.savefig(f'attns_fig/precision_sz{mask_d}_mode_{pos}_total.png',dpi=300)
#     # plt.figure(figsize=(5,5))
# # plt.imshow(zscore.T,alpha=1)

In [ ]:

def write_vtp(outpd,mask_id,name='tract'):
    path = pathlib.Path(f'tracts')
    path.mkdir(parents=True, exist_ok=True)
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetDataModeToBinary()
    outlines = vtk.vtkCellArray()
    outpoints = vtk.vtkPoints()
    polydata = vtk.vtkPolyData()
    outlines.InitTraversal()
    for i in mask_id.squeeze():
        # if 'T_Sup_left' in str(tract_name[tract_list[i].astype(int)]):
        if str(tract_name[tract_list[i].astype(int)]) == str(name):
            cellptids = vtk.vtkIdList()
            for j in range(outpd.GetCell(i).GetNumberOfPoints()):
                idx = outpoints.InsertNextPoint(outpd.GetPoint(outpd.GetCell(i).GetPointId(j)))
                cellptids.InsertNextId(idx)
            outlines.InsertNextCell(cellptids)
        
    polydata.SetLines(outlines)
    polydata.SetPoints(outpoints)
    writer.SetInputData(polydata)
    name = name.split('\'')[1]
    writer.SetFileName(path/(name+'.vtp'))
    writer.Update()
for name in np.unique(tract_name[tract_list[tracts_id].astype(int)]):
    write_vtp(outpd,tracts_id,str(name))

In [ ]:
ax = plt.subplot(2,2,1)
ax.imshow(data[...,0])
ax = plt.subplot(2,2,2)
ax.imshow(data[...,1])
ax = plt.subplot(2,2,3)
ax.imshow(data[...,2])
ax = plt.subplot(2,2,4)
ax.imshow(data.sum(-1))

In [ ]:
from pathlib import Path
import pandas as pd
import re
fold = pd.read_csv('fold.csv')
def extract_acc(path='/data04/junyi/results/mvit_nonorm_balanced_fold'):
    a = {}
    aug_num_lst = []
    for path in sorted(Path(path).glob('**/*.log')):
        pattern = re.findall(r"[^_]+",path.parent.name)
        # print(pattern)
        aug_num = int(pattern[-2])
        # print(path.parent.name.rsplit('_'))
        with open(path,mode='r') as f:
            for line in f:
                if ('The Final Min Err of fold') in line:
            
                    fold_num,acc = line.strip().rsplit('The Final Min Err of fold')[-1].split(':')
                    num = (fold['fold'] == int(fold_num)).sum()
                    print(num,fold_num)
                    if int(fold_num) in a:
                        a[int(fold_num)].append(float(acc)*num/len(fold))
                    else:
                        a[int(fold_num)] = [float(acc)*num/len(fold)]
                    break
        aug_num_lst.append(aug_num)
    res = []
    [res.append(x) for x in aug_num_lst if x not in res]
    df = pd.DataFrame(a)
    df.index = [f'Aug:{i}'for i in res]
    df.sort_index(inplace=True)
    return df

In [ ]:
for path in sorted(Path('/data06/junyi/results').iterdir()):
    try:
        if path.is_dir():
            df = extract_acc(path)
            print(f'Name:{path.name}\nACC:\n{100-df.sum(axis=1)}')
    except:
        print(f'Name:{path.name}\nACC:Error')

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
df = pd.read_csv('/data01/zixi/TractoFormer/TractoFormer-MVIT-main/HCP_1000.csv')


In [12]:
%env CUDA_VISIBLE_DEVICES=0,3
# %env LD_LIBRARY_PATH = ''
try:
  from typing import Literal
  ModelName = Literal["MViTv2-B", "MViTv2-S", "MViTv2-L", "MViTv2-T", ]
except ImportError:
  ModelName = str
import torch
import torchvision.transforms.functional as F
import numpy as np
from mvit.datasets.tractoembedding import Tractoembedding

gpus_str = ''
gpus_list = []
# for i in range(torch.cuda.device_count()):
#     used,all=torch.cuda.mem_get_info(device=i)
#     if used/all >=0.80:
#         gpus_str+=f'{i},'
#         gpus_list += [i]
# os.environ['CUDA_VISIBLE_DEVICES'] = gpus_str[:-1]
# world_size = num_proc * num_shards
# rank = shard_id * num_proc + local_rank
from matplotlib import pyplot as plt
def load_model_and_preproc(model="MViTv2-mri", use_cuda:bool=torch.cuda.is_available(), use_half:bool=True):
    """
    Load an MViT2 model and get the image transformer
    :param model: the name of the model. Should be  one of "MViTv2-B", "MViTv2-S", "MViTv2-L", "MViTv2-T". Defaults to "MViTv2-B".
    :param use_cuda: whether to use cuda. Defaults to True if cuda is available.
    :param use_half: whether to use half-precision. Defaults to True, but will not be used if cuda is not used.
    :returns a tuple of (the model, image transformer)
    """
    model = model.replace('-','_')
    use_half &= use_cuda

    import urllib.request
    from mvit.config.defaults import get_cfg
    from mvit.models import build_model
    from mvit.datasets import Tractoembedding
    from mvit.datasets import loader
    mode = 'fusion'#多模态融合模式，FA1,density，trace1
    ckpt_path = '/data01/zixi/TractoFormer/TractoFormer-MVIT-main/synthetic_v1_0/checkpoints/checkpoint_epoch_00140.pyth'
    # ckpt_path = f"/data04/junyi/results/mvit_balanced_fold_new_norm_direct/{mode}_3_030_0123/checkpoints/checkpoint_epoch_00160.pyth"
    # if not os.path.exists(ckpt_path):
    #     urllib.request.urlretrieve(f"https://dl.fbaipublicfiles.com/mvit/mvitv2_models/{ckpt_path}", ckpt_path)
    cfg = get_cfg()
    
    # cfg.MVIT.CLS_EMBED_ON = True
    cfg.DATA.PATH_TO_DATA_DIR = '/data01/zixi/synthetic_noise/selected_100_males.csv'
    # cfg.merge_from_file(f"./configs/test/{model.replace('ViT','VIT')}_test.yaml")
    cfg.merge_from_file('configs/MVITv2_mri.yaml')
    cfg.DATA.MODE = 'fusion'
    cfg.MVIT.CLS_EMBED_ON = True
    cfg.DATA_AUG_NUM =1
    cfg.DATA_NUM = 3
    cfg.NUM_GPUS = 1
    cfg.DATA_MODE = mode
    cfg.TRAIN.BATCH_SIZE = 4

    model = build_model(cfg)
    ckpt = torch.load(ckpt_path)
    model.load_state_dict(ckpt["model_state"])
    
    model.eval()
    # if use_half:
    #     model = model.half()
    return model,loader.construct_loader(cfg, "val",0)
    
model, datagen = load_model_and_preproc()
datagen.dataset.return_subid = True
input_images, labels, sub_ids = next(iter(datagen))
print("当前 batch 的脑 ID:", sub_ids)

env: CUDA_VISIBLE_DEVICES=0,3


/tmp/ipykernel_1920764/1743459474.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path)


<class 'mvit.datasets.tractoembedding.Tractoembedding'>
(100, 3)
当前 batch 的脑 ID: tensor([196851, 130720, 175035, 158136])


In [13]:
ckpt = torch.load('/data01/zixi/TractoFormer/TractoFormer-MVIT-main/synthetic_v1_0/checkpoints/checkpoint_epoch_00140.pyth', map_location='cpu')
print([k for k in ckpt["model_state"].keys() if "blocks" in k][:20])


/tmp/ipykernel_1920764/4015468948.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('/data01/zixi/TractoFormer/TractoFormer-MVIT-main/synthetic_v1_0/chec

['model_list.0.blocks.0.norm1.weight', 'model_list.0.blocks.0.norm1.bias', 'model_list.0.blocks.0.attn.rel_pos_h', 'model_list.0.blocks.0.attn.rel_pos_w', 'model_list.0.blocks.0.attn.qkv.weight', 'model_list.0.blocks.0.attn.qkv.bias', 'model_list.0.blocks.0.attn.proj.weight', 'model_list.0.blocks.0.attn.proj.bias', 'model_list.0.blocks.0.attn.pool_q.weight', 'model_list.0.blocks.0.attn.norm_q.weight', 'model_list.0.blocks.0.attn.norm_q.bias', 'model_list.0.blocks.0.attn.pool_k.weight', 'model_list.0.blocks.0.attn.norm_k.weight', 'model_list.0.blocks.0.attn.norm_k.bias', 'model_list.0.blocks.0.attn.pool_v.weight', 'model_list.0.blocks.0.attn.norm_v.weight', 'model_list.0.blocks.0.attn.norm_v.bias', 'model_list.0.blocks.0.norm2.weight', 'model_list.0.blocks.0.norm2.bias', 'model_list.0.blocks.0.mlp.fc1.weight']


In [ ]:
# input_images, labels, sub_ids = next(iter(datagen))
# print("当前 batch 的脑 ID:", sub_ids)
# # input_images, labels= next(iter(datagen))


In [ ]:
# for input_images,labels in datagen:
#     img = input_images[0][0][0]
#     mask = img!=0
#     # mt = masked_tensor(img,img!=0)
#     mean = img.mean((2,3),  keepdim=True).mean(0,keepdim=True)
#     std = img.std((2,3),keepdim=True).mean(0,keepdim=True)
#     img1 = ((img-mean)/std)
#     # img1[~mask] = 0
#     print(mean.squeeze(),std.squeeze())
    

In [14]:
allimages = []
for images4model in input_images[0]:
    imagecuda = [image.cuda() for image in images4model]
    allimages.append(imagecuda)
print(f"PyTorch sees {torch.cuda.device_count()} GPU(s).")
print(f"Using device: {torch.cuda.current_device()} → {torch.cuda.get_device_name(torch.cuda.current_device())}")


PyTorch sees 2 GPU(s).
Using device: 0 → NVIDIA GeForce RTX 3090


In [15]:

with torch.no_grad():   
    model.eval()
    preds = model(allimages)
mixed = [((labels==0)&(preds==labels)).float().sum(),
                 ((labels==0)&(preds!=labels)).float().sum(),
                 ((labels==1)&(preds==labels)).float().sum(),
                 ((labels==1)&(preds!=labels)).float().sum()]
prec = mixed[0]/(mixed[0]+mixed[1])
recall = mixed[0]/(mixed[0]+mixed[3])
f1 = 2*prec*recall/(prec+recall)

RuntimeError: The size of tensor a (2) must match the size of tensor b (4) at non-singleton dimension 1

In [ ]:
# #叠加到fa image上面的
# from itertools import product
# import pathlib
# import torch
# from torchvision.transforms import functional as F
# import matplotlib.pyplot as plt
# import numpy as np
# import h5py
# import nibabel as nib

# # ===================== 基础参数 =====================
# precison_list = ['sz80', 'sz160', 'sz320'][::-1]
# mode_list = ['Left', 'Right', 'Commisural']

# # 获取预测正确的 sample mask
# v, i = torch.topk(preds, 1)
# labels = labels.cuda()
# m = (labels == i.squeeze())
# print("匹配结果 m:", m)

# # ===================== 获取真实 sub_id =====================
# sub_id = sub_ids[0].item() if torch.is_tensor(sub_ids[0]) else sub_ids[0]
# root = f"/data01/zixi/tractoembedding_PPMI_143/{sub_id}"
# print(f"当前可视化的 subject: {sub_id}")

# # ===================== tract 信息（自动） =====================
# h5_path = f"{root}/tracts/{sub_id}.h5"
# with h5py.File(h5_path, "r") as f:
#     tract_name = f['tract_name'][:]
# print("Tract 名称数量:", len(np.unique(tract_name)))

# # ===================== 参考影像（FA） =====================
# fa_path = f"{root}/tractoembedding/da-full/{sub_id}-FA1_CLR_sz640.nii.gz"
# fa_img = nib.load(fa_path)
# fa_data = fa_img.get_fdata()
# affine = fa_img.affine
# inv_affine = np.linalg.inv(affine)
# X, Y, Z = fa_data.shape

# print(f"[FA] 体素形状: {fa_data.shape}, 仿射:\n{affine}")

# # 可视化平面与切片
# plane = "axial"
# slice_idx = max(1, Z // 2)


# # ===================== Heatmap 上采样 =====================
# def overlay_on_slice(attn_20x20, volume, plane, slice_index):
#     if plane == "axial":
#         base_slice = volume[:, :, slice_index]
#         target_hw = (base_slice.shape[1], base_slice.shape[0])
#     else:
#         raise ValueError("仅实现 axial 模式")

#     attn_t = torch.tensor(attn_20x20, dtype=torch.float32).unsqueeze(0)
#     attn_rs = F.resize(attn_t, target_hw, F.InterpolationMode.BILINEAR)[0].numpy()
#     return base_slice, attn_rs


# # ===================== 绘图（无 fiber，只 attention） =====================
# def plot_slice(base_slice, attn_rs, title, save_path):
#     fig, ax = plt.subplots(figsize=(7, 7))
#     ax.imshow(base_slice.T, cmap='gray', alpha=0.75, origin='lower')
#     ax.imshow(attn_rs.T, cmap='jet', alpha=0.35, origin='lower', vmin=0, vmax=0.05)

#     ax.set_title(title, fontsize=12)
#     ax.axis('off')
#     fig.tight_layout()
#     fig.savefig(save_path, dpi=300)
#     plt.close()


# # ===================== 生成可视化 =====================
# out_dir = pathlib.Path("./attns_only_attn")
# out_dir.mkdir(exist_ok=True)

# for mi, pi in product(range(3), range(3)):
#     attns = model.model_list[mi].blocks[2*pi].attn.attn_lst[m, :, 0, 1:].mean(0, keepdim=True)
#     B, nH, nP = attns.shape
#     for head in range(nH):
#         attn_20 = attns[0, head].reshape(20, 20).cpu().numpy()
#         base_slice, attn_rs = overlay_on_slice(attn_20, fa_data, plane, slice_idx)

#         title = f"Sub: {sub_id} | Prec: {precison_list[pi]}, Mode: {mode_list[mi]} | z={slice_idx}, Head={head}"
#         save_path = out_dir / f"{sub_id}_prec_{precison_list[pi]}_mode_{mode_list[mi]}_{plane}_z{slice_idx}_head{head}.png"

#         plot_slice(base_slice, attn_rs, title, save_path)

# print("🎉 已完成：纯 attention map 可视化（无 fiber）")


In [16]:
#80*80s升采样到160*160， 320*320降采样到160*160
import os
import nibabel as nib
import numpy as np
import torch
import torch.nn.functional as F

# ---------------------------
# 函数：把单通道图像 resize 到 160×160
# ---------------------------
def resize_to_160(img_2d):
    t = torch.tensor(img_2d).unsqueeze(0).unsqueeze(0)  # (1,1,H,W)
    t_res = F.interpolate(
        t, size=(160,160), mode="bilinear", align_corners=False
    )[0,0].numpy()
    return t_res


# ---------------------------
# 函数：按 C/L/R 通道分别统一到 160×160，并求 mean
# ---------------------------
def merge_channels(sub_id):
    root = f"/data01/zixi/synthetic_noise/synthetic_embedding/{sub_id}/tractoembedding_noisy/tractoembedding/da-full"

    paths = {
        "80":  f"{root}/{sub_id}-FA1_CLR_sz80.nii.gz",
        "160": f"{root}/{sub_id}-FA1_CLR_sz160.nii.gz",
        "320": f"{root}/{sub_id}-FA1_CLR_sz320.nii.gz",
    }

    print("===== 检查输入文件 =====")
    available = []
    for k, p in paths.items():
        print(f"检查 {p}")
        if os.path.exists(p):
            print("  ✔ 找到文件")
            available.append((k, p))
        else:
            print("  ❌ 文件不存在")

    if not available:
        print("❌ 没有可用输入文件，退出")
        return

    print(f"\n📌 处理 subject {sub_id}，找到 {len(available)} 个有效输入文件\n")

    # (160,160,3) → [Comm, Left, Right]
    merged = np.zeros((160,160,3), dtype=np.float32)

    for ch in range(3):
        print(f"\n====== 处理通道 {ch} ======")
        resized_imgs = []

        for size_label, file_path in available:
            img = nib.load(file_path)
            data = img.get_fdata().astype(np.float32)

            channel_img = data[:, :, ch]
            print(f"{size_label} 通道 {ch} 原始 shape: {channel_img.shape}")

            resized = resize_to_160(channel_img)
            print(f"{size_label} 通道 {ch} resize 后 shape: {resized.shape}")

            resized_imgs.append(resized)

        merged[:,:,ch] = np.mean(resized_imgs, axis=0)
        print(f"⭐ 通道 {ch} 合并完成, mean 后 min={merged[:,:,ch].min()}, max={merged[:,:,ch].max()}")

    mean_dir = os.path.join(root, "mean")
    os.makedirs(mean_dir, exist_ok=True)

    out_path = os.path.join(mean_dir, f"{sub_id}-FA1_CLR_sz160_merged.nii.gz")
    nib.save(nib.Nifti1Image(merged, np.eye(4)), out_path)

    print(f"\n🎉 FINISHED! 文件保存于:")
    print(out_path)


# ---------------------------
# 实际执行
# ---------------------------
merge_channels("196851")


===== 检查输入文件 =====
检查 /data01/zixi/synthetic_noise/synthetic_embedding/196851/tractoembedding_noisy/tractoembedding/da-full/196851-FA1_CLR_sz80.nii.gz
  ✔ 找到文件
检查 /data01/zixi/synthetic_noise/synthetic_embedding/196851/tractoembedding_noisy/tractoembedding/da-full/196851-FA1_CLR_sz160.nii.gz
  ✔ 找到文件
检查 /data01/zixi/synthetic_noise/synthetic_embedding/196851/tractoembedding_noisy/tractoembedding/da-full/196851-FA1_CLR_sz320.nii.gz
  ✔ 找到文件

📌 处理 subject 196851，找到 3 个有效输入文件


====== 处理通道 0 ======
80 通道 0 原始 shape: (80, 80)
80 通道 0 resize 后 shape: (160, 160)
160 通道 0 原始 shape: (160, 160)
160 通道 0 resize 后 shape: (160, 160)
320 通道 0 原始 shape: (320, 320)
320 通道 0 resize 后 shape: (160, 160)
⭐ 通道 0 合并完成, mean 后 min=-0.07518885284662247, max=0.821058452129364

====== 处理通道 1 ======
80 通道 1 原始 shape: (80, 80)
80 通道 1 resize 后 shape: (160, 160)
160 通道 1 原始 shape: (160, 160)
160 通道 1 resize 后 shape: (160, 160)
320 通道 1 原始 shape: (320, 320)
320 通道 1 resize 后 shape: (160, 160)
⭐ 通道 1 合并完成, mean 后 m

In [ ]:
#使用threshold圈轮廓
from itertools import product
import pathlib
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from skimage import measure

# ===================== 基础参数 =====================
mode_list = ['Left', 'Right', 'Commisural']
resolutions = ['sz80', 'sz160', 'sz320']
pi_map = {'sz80':2, 'sz160':1, 'sz320':0}

# ===================== 正确 sample =====================
v, i = torch.topk(preds, 1)
labels = labels.cuda()
m = (labels == i.squeeze())

# ===================== sub_id =====================
sub_id = sub_ids[0].item() if torch.is_tensor(sub_ids[0]) else sub_ids[0]
root = f"/data01/zixi/synthetic_noise/synthetic_embedding/{sub_id}"

# ===================== 背景（160×160×3） =====================
bg_path = f"{root}/tractoembedding_noisy/tractoembedding/da-full/mean/{sub_id}-FA1_CLR_sz160_merged.nii.gz"
bg_data = nib.load(bg_path).get_fdata().astype(np.float32)

H, W, C = bg_data.shape

# ===================== resize 20→160 =====================
def resize20_to_160(attn20):
    t = torch.tensor(attn20).unsqueeze(0).unsqueeze(0)
    return F.interpolate(t, size=(H,W), mode="bilinear", align_corners=False)[0,0].numpy()


# ===================== Hotspot 可视化（P95 阈值 + 不同颜色） =====================
def plot_hotspot(base, attn160, title, path, color='red', p=99):
    """
    base    : (H,W) 背景 FA 单通道
    attn160 : (H,W) attention（上采样后）
    color   : 轮廓颜色 ('red','blue','green')
    p       : 百分位阈值 (默认 95)
    """
    # 1. 阈值
    #attn160 = attn160.copy()
    #attn160[:, 140:160] = 0   # 把顶部 20 列干掉

    T = np.percentile(attn160, p)
    print(f"使用 P{p} 阈值 = {T:.5f}")

    mask = attn160 > T
    mask_disp = mask.T

    # 2. 寻找轮廓
    contours = measure.find_contours(mask_disp.astype(float), 0.5)

    # 3. 绘图
    fig, ax = plt.subplots(figsize=(7, 7))

    ax.imshow(base.T, cmap='gray', alpha=0.85, origin='lower')
    ax.imshow(attn160.T, cmap='jet',  alpha=0.45, origin='lower')

    for cnt in contours:
        ax.plot(cnt[:, 1], cnt[:, 0], color=color, linewidth=2)

    ax.set_title(title)
    ax.axis('off')
    fig.tight_layout()
    fig.savefig(path, dpi=300)
    plt.close()


# ===================== 输出目录 =====================
out_dir = pathlib.Path(f"{root}/attns_merged_hotspot_P99")
out_dir.mkdir(exist_ok=True)


# ===================== 主循环 =====================
for mi in range(3):

    nH = model.model_list[mi].blocks[0].attn.num_heads

    for head in range(nH):

        # 三分辨率 attention → 160×160
        attn_160_list = []
        for res in resolutions:
            pi = pi_map[res]
            att = model.model_list[mi].blocks[2*pi].attn.attn_lst[m, :, 0, 1:].mean(0)
            att20 = att[head].reshape(20,20).cpu().numpy()
            attn_160_list.append(resize20_to_160(att20))

        # merge
        attn_merge_160 = np.maximum.reduce(attn_160_list)
        np.save(out_dir / f"{sub_id}_MI{mi}_HEAD{head}_attn160.npy", attn_merge_160)

        # 不同背景对应不同颜色
        color_map = {
            "Commisural": "red",
            "Left": "blue",
            "Right": "green"
        }

        for bg_ch, bg_name in zip([0,1,2], ["Commisural","Left","Right"]):

            base_slice = bg_data[:,:,bg_ch]

            save_path = out_dir / f"{sub_id}_MI{mi}_HEAD{head}_BG{bg_name}_hotP95.png"
            title = f"{sub_id} | MI{mi} | Head{head} | BG={bg_name} | hotspot(P95)"

            plot_hotspot(
                base_slice,
                attn_merge_160,
                title,
                save_path,
                color=color_map[bg_name],
                p=99   # ← 这里改成 95
            )

print("🎉 完成：Hotspot (P99) 可视化")


In [17]:
#使用threshold圈轮廓
from itertools import product
import pathlib
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from skimage import measure

# ===================== 基础参数 =====================
mode_list = ['Left', 'Right', 'Commisural']
resolutions = ['sz80', 'sz160', 'sz320']
pi_map = {'sz80':2, 'sz160':1, 'sz320':0}

# ===================== 正确 sample（修复：只取一个样本） =====================
v, i = torch.topk(preds, 1)
labels = labels.cuda()
m = (labels == i.squeeze())

good_idx = m.nonzero(as_tuple=True)[0]
if len(good_idx) > 0:
    idx = good_idx[0].item()     # 取第 1 个预测正确的样本
else:
    idx = 0                      # 若全错，就 fallback 到 batch 第一个样本

print(f"📌 使用 sample index: {idx} (是否正确: {m[idx].item()})")

# ===================== sub_id =====================
sub_id = sub_ids[0].item() if torch.is_tensor(sub_ids[0]) else sub_ids[0]
root = f"/data01/zixi/synthetic_noise/synthetic_embedding/{sub_id}"

# ===================== 背景（160×160×3） =====================
bg_path = f"{root}/tractoembedding_noisy/tractoembedding/da-full/mean/{sub_id}-FA1_CLR_sz160_merged.nii.gz"
bg_data = nib.load(bg_path).get_fdata().astype(np.float32)

H, W, C = bg_data.shape

# ===================== resize 20→160 =====================
def resize20_to_160(attn20):
    attn20 = np.nan_to_num(attn20, nan=0.0, posinf=0.0, neginf=0.0)
    t = torch.tensor(attn20).unsqueeze(0).unsqueeze(0)
    return F.interpolate(t, size=(H,W), mode="bilinear", align_corners=False)[0,0].numpy()

# ===================== Hotspot 可视化 =====================
def plot_hotspot(base, attn160, title, path, color='red', p=99):

    # 处理 NaN
    attn160 = np.nan_to_num(attn160, nan=0.0, posinf=0.0, neginf=0.0)

    # 全零直接跳过
    if not np.any(attn160):
        print(f"⚠️ {title}: attention map 全 0，跳过绘制")
        return

    # P99 阈值（nan-safe）
    T = np.nanpercentile(attn160, p)
    print(f"使用 P{p} 阈值 = {T:.5f}")

    mask = attn160 > T
    mask_disp = mask.T

    # 寻找轮廓
    contours = measure.find_contours(mask_disp.astype(float), 0.5)

    # 画图
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.imshow(base.T, cmap='gray', alpha=0.85, origin='lower')
    ax.imshow(attn160.T, cmap='jet',  alpha=0.45, origin='lower')

    for cnt in contours:
        ax.plot(cnt[:, 1], cnt[:, 0], color=color, linewidth=2)

    ax.set_title(title)
    ax.axis('off')
    fig.tight_layout()
    fig.savefig(path, dpi=300)
    plt.close()

# ===================== 输出目录 =====================
out_dir = pathlib.Path(f"{root}/attns_merged_hotspot_P99")
out_dir.mkdir(exist_ok=True)

# ===================== 主循环 =====================
for mi in range(3):

    nH = model.model_list[mi].blocks[0].attn.num_heads

    for head in range(nH):

        attn_160_list = []

        # 三分辨率 attention → 160×160
        for res in resolutions:
            pi = pi_map[res]

            # ⭐ 修复：从第 idx 个样本取 attention，而不是用 m 做 mask
            att = model.model_list[mi].blocks[2*pi].attn.attn_lst[idx, :, 0, 1:]

            att20 = att[head].reshape(20,20).cpu().numpy()
            attn_160_list.append(resize20_to_160(att20))

        # merge
        attn_merge_160 = np.maximum.reduce(attn_160_list)
        attn_merge_160 = np.nan_to_num(attn_merge_160, nan=0.0)

        np.save(out_dir / f"{sub_id}_MI{mi}_HEAD{head}_attn160.npy", attn_merge_160)

        color_map = {"Commisural": "red", "Left": "blue", "Right": "green"}

        for bg_ch, bg_name in zip([0,1,2], ["Commisural","Left","Right"]):

            base_slice = bg_data[:,:,bg_ch]

            save_path = out_dir / f"{sub_id}_MI{mi}_HEAD{head}_BG{bg_name}_hotP99.png"
            title = f"{sub_id} | MI{mi} | Head{head} | BG={bg_name} | hotspot(P99)"

            plot_hotspot(
                base_slice,
                attn_merge_160,
                title,
                save_path,
                color=color_map[bg_name],
                p=99
            )

print("🎉 完成：Hotspot (P99) 可视化（修复版）")


📌 使用 sample index: 0 (是否正确: False)
使用 P99 阈值 = 0.02659
使用 P99 阈值 = 0.02659
使用 P99 阈值 = 0.02659
使用 P99 阈值 = 0.02420
使用 P99 阈值 = 0.02420
使用 P99 阈值 = 0.02420
使用 P99 阈值 = 0.00645
使用 P99 阈值 = 0.00645
使用 P99 阈值 = 0.00645
🎉 完成：Hotspot (P99) 可视化（修复版）


In [ ]:
#生成left/right/commisural tract attention 排名
import os
import numpy as np
import h5py

# ============================================
# (1) embed.npy → 20×20 pixel  ← 注意：不是 160 ！
# ============================================
def embed_to_pixel_20(embed):
    """
    embed.npy shape = (N, 2)
    注意：TractoEmbed 的 embed 真实范围约 [-2.5, 3.2]
    对 20×20 grid 的映射如下：
        pixel = round( (embed - min) / (max - min) * 19 )
    """
    x = embed[:,0]
    y = embed[:,1]

    xmin, xmax = x.min(), x.max()
    ymin, ymax = y.min(), y.max()

    # normalize to [0, 19]
    px = np.round((x - xmin) / (xmax - xmin) * 19).astype(int)
    py = np.round((y - ymin) / (ymax - ymin) * 19).astype(int)

    px = np.clip(px, 0, 19)
    py = np.clip(py, 0, 19)

    return np.stack([px, py], axis=1)


# ============================================
# (2) tract 左/右/胼胝体 分区
# ============================================
def compute_fiber_location(tract_list, tract_name):
    N = tract_list.shape[0]
    loc = np.zeros(N, dtype=np.int32)
    max_tid = len(tract_name) - 1

    # ---- convert to safe integer tract id ----
    tract_ids = tract_list.astype(np.int32)   # float → int

    # ---- clamp invalid values ----
    tract_ids[tract_ids < 0] = 0
    tract_ids[tract_ids > max_tid] = 0

    for i in range(N):
        tid = tract_ids[i]

        tname = tract_name[tid]
        tname = tname.decode() if isinstance(tname, bytes) else tname
        tname = tname.lower()

        if "left" in tname:
            loc[i] = 1
        elif "right" in tname:
            loc[i] = 2
        elif "comm" in tname:
            loc[i] = 0
        else:
            loc[i] = 0

    return loc



# ============================================
# (3) fiber-level attention
# ============================================
def compute_fiber_attention(att20, fiber_pix):
    """
    att20 : (20,20) numpy array
    fiber_pix : (N,2)
    """
    px = fiber_pix[:,0]
    py = fiber_pix[:,1]
    return att20[px, py]   # (N,)


# ============================================
# (4) 按 tract 聚合 attention
# ============================================
def aggregate_by_tract(fiber_attn, tract_list, tract_name, fiber_loc, hemi_code):
    """
    fiber_attn : 每条 fiber 的 attention (N,)
    tract_list : N 个属于 0~57 的 tract id（可能有非法值）
    tract_name : 58 个 tract 名称
    """

    N = len(fiber_attn)
    max_tid = len(tract_name) - 1

    # ---- 本半球 fiber ----
    idx = np.where(fiber_loc == hemi_code)[0]
    if len(idx) == 0:
        return []

    att = fiber_attn[idx]
    tids = tract_list[idx]

    tract_scores = {}

    for tid, score in zip(tids, att):

        # ========== 🛡 强制安全过滤 ==========
        if np.isnan(tid):
            tid = 0
        else:
            tid = int(tid)

        if tid < 0 or tid > max_tid:
            tid = 0
        # ====================================

        if tid not in tract_scores:
            tract_scores[tid] = []
        tract_scores[tid].append(score)

    # ---- 聚合均值并读取 tract_name ----
    result = []
    for tid, scores in tract_scores.items():
        name = tract_name[tid]
        name = name.decode() if isinstance(name, bytes) else name
        result.append((name, np.mean(scores)))

    # attention 从大到小排序
    return sorted(result, key=lambda x: -x[1])



# ============================================
# (5) 主流程：真正 fiber-level attention 分析
# ============================================
def run_fiber_level(sub_id):

    root = f"/data01/zixi/tractoembedding_PPMI_143/{sub_id}"

    # 读取 embed
    embed = np.load(f"{root}/embed.npy")
    fiber_pix20 = embed_to_pixel_20(embed)

    # tract 信息
    with h5py.File(f"{root}/tracts/{sub_id}.h5", "r") as f:
        tract_list = f["tract_list"][:]
        tract_name = f["tract_name"][:]

    fiber_loc = compute_fiber_location(tract_list, tract_name)

    # 读取三个 MI 对应的 attention （20×20 原图）
    att_files = {
        "Left":       f"{root}/attns_merged_hotspot_P99/{sub_id}_MI0_HEAD0_attn160.npy",
        "Right":      f"{root}/attns_merged_hotspot_P99/{sub_id}_MI1_HEAD0_attn160.npy",
        "Commisural": f"{root}/attns_merged_hotspot_P99/{sub_id}_MI2_HEAD0_attn160.npy",
    }

    out_dir = f"{root}/fiber_results_fiberLevel"
    os.makedirs(out_dir, exist_ok=True)

    for region, fpath in att_files.items():

        # --- 原始 att20 获取 ---
        # 你保存的是 160，但我们从模型直接取 20×20 更好
        # 为简单起见，这里从 160 降采样回 20
        att160 = np.load(fpath)
        att20 = att160.reshape(20,8,20,8).mean(axis=(1,3))  # block average to 20×20

        fiber_att = compute_fiber_attention(att20, fiber_pix20)

        hemi = {"Left":1, "Right":2, "Commisural":0}[region]

        tracts_rank = aggregate_by_tract(
            fiber_att, tract_list, tract_name, fiber_loc, hemi
        )

        # 写入 txt
        txt = f"{out_dir}/{sub_id}_{region}.txt"
        with open(txt, "w") as f:
            for name, score in tracts_rank:
                f.write(f"{score:.6f}  {name}\n")

        print(f"✔ 写入：{txt}")

    print("\n🎉 fiber-level attention 完成！")


# --------------------------------------------
# 执行
# --------------------------------------------
run_fiber_level("129634")


In [ ]:
#直接进行multi-scale的版本
from itertools import product
import pathlib
import torch
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt

precison_list = ['sz80', 'sz160', 'sz320',]
precison_list.reverse()
mode_list = ['Left', 'Right', 'Commisural']

v, i = torch.topk(preds, 1)
labels = labels.cuda()
m = labels == i.squeeze()
print(m)


import h5py

f = h5py.File('/data01/zixi/synthetic_noise/synthetic_embedding/196851/tractoembedding_noisy/tracts/196851.h5', "r")
tract_list = f['tract_list'][:]
tract_name = f['tract_name'][:]
print(len(np.unique(tract_name)))

# ✅ 只加这一行，假设你有 sub_ids 变量
sub_id = sub_ids[0].item() if torch.is_tensor(sub_ids[0]) else sub_ids[0]

for mi, pi in product(range(3), range(3)):
    attns = model.model_list[mi].blocks[2*pi].attn.attn_lst[m, :, 0, 1:].mean(0, keepdim=True)
    B, nH, nP = attns.shape
    fig = plt.figure(figsize=(8, 5))
    for head in range(nH):
        attn = attns[0, head].reshape(-1, 20, 20).cpu()
        mask = F.resize(attn, input_images[0][0][0].shape[-2:], F.InterpolationMode.BILINEAR)
        # 保存为 .npy 文件
        npy_path = f'./attns/{sub_id}_precision_{precison_list[pi]}_mode_{mode_list[mi]}_head{head}.npy'
        np.save(npy_path, mask.numpy())
        
        embed_feat = np.load('/data01/zixi/synthetic_noise/synthetic_embedding/196851/tractoembedding_noisy/embed_feat.npy')
        embed = np.load('/data01/zixi/synthetic_noise/synthetic_embedding/196851/tractoembedding_noisy/embed.npy')
        
        print(embed.shape, embed_feat.shape)

        true_x, true_y = np.where(mask[0] > 0.05)
        indice = true_x + true_y * mask.shape[-1]
        
        fiber_indices = np.load('/data01/zixi/synthetic_noise/synthetic_embedding/196851/tractoembedding_noisy/tractoembedding/da-full/fiber_indices.npy')
        true_fiber_indices = fiber_indices[indice]
        
        print(np.unique(tract_name[tract_list[true_fiber_indices].astype(int)]))
        
        

        ax = plt.subplot(1, nH, head + 1, frameon=False)
        ax.imshow(mask[0], vmin=0, vmax=0.05, cmap='jet')
        ax.axis('off')
        ax.set_title(f'Head: {head}', fontsize=8)
    
    # ✅ 在这里标题中加上 ID
    fig.suptitle(
        f'Subject ID: {sub_id} | Precision: {precison_list[pi]}, Mode: {mode_list[mi]}',
        fontsize=10, fontweight='bold', y=0.95
    )
    
    fig.tight_layout()
    pathlib.Path('./attns').mkdir(exist_ok=True)
    fig.savefig(f'./attns/precision_{precison_list[pi]}_mode_{mode_list[mi]}.png', dpi=300)

In [ ]:
import numpy as np
import h5py
import nibabel as nib

# ===================== sub_id =====================
sub_id = int(sub_ids[0])
root = f"/data01/zixi/tractoembedding_PPMI_143/{sub_id}"

print("=== Checking files under:", root, " ===")

# ===================== fiber_indices =====================
fiber_indices_path = f"{root}/tractoembedding/da-full/fiber_indices.npy"
fiber_indices = np.load(fiber_indices_path)
print("\nLoaded fiber_indices:", fiber_indices_path)

# ===================== tract_list & tract_name =====================
h5_path = f"{root}/tracts/{sub_id}.h5"
with h5py.File(h5_path, "r") as f:
    tract_list = f["tract_list"][:]
    tract_name = f["tract_name"][:]

print("Loaded tract_list & tract_name:", h5_path)

# ===================== Debug 输出 =====================

print("\n========== DEBUG OUTPUT ==========")
print("fiber_indices shape =", fiber_indices.shape)
print("fiber_indices dtype =", fiber_indices.dtype)

print("min =", fiber_indices.min(), 
      " max =", fiber_indices.max())

print("\ntract_list shape =", tract_list.shape)
print("tract_list dtype =", tract_list.dtype)

print("tract_name shape =", tract_name.shape)

# 检查 tract_list 是否有非法 index（超出 tract_name 范围）
bad_tract_ids = tract_list[(tract_list < 0) | (tract_list >= len(tract_name))]

print("\n非法 tract_id 个数 =", len(bad_tract_ids))
if len(bad_tract_ids) > 0:
    print("前 20 个非法 tract_id:", bad_tract_ids[:20])

print("========== END DEBUG ==========\n")


In [ ]:
import numpy as np
import pathlib
import h5py
import nibabel as nib

# ===================== sub_id =====================
sub_id = int(sub_ids[0])
root = f"/data01/zixi/tractoembedding_PPMI_143/{sub_id}"

# ===================== load fiber info =====================
h5_path = f"{root}/tracts/{sub_id}.h5"
with h5py.File(h5_path, "r") as f:
    tract_list = f["tract_list"][:]
    tract_name = f["tract_name"][:]

fiber_indices = np.load(f"{root}/tractoembedding/da-full/fiber_indices.npy")

# ===================== load one merged embedding to get size =====================
emb_path = f"{root}/tractoembedding/da-full/mean/{sub_id}-FA1_CLR_sz160_merged.nii.gz"
emb = nib.load(emb_path)
H, W, _ = emb.shape

# ===================== threshold =====================
def threshold_p97(attn160):
    vals = attn160[attn160 > 0]
    if vals.size == 0:
        return 999
    return np.percentile(vals, 97)

# ===================== hotspot → fibers =====================
def extract_fibers(attn160, T):

    mask = attn160 > T
    yy, xx = np.where(mask)
    if len(xx) == 0:
        return []

    # 160×160 → 20×20
    px = (xx * 20 / H).astype(int)
    py = (yy * 20 / W).astype(int)
    patch = px + py * 20

    ids = fiber_indices[patch]

    names = []
    for k in ids:
        name = tract_name[tract_list[k]]
        if isinstance(name, bytes):
            name = name.decode()
        names.append(name)

    return np.unique(names)

# ===================== input folder: (你已经生成的 hotspot npy) =====================
attn_folder = pathlib.Path(f"{root}/attns_merged_hotspot_P97")

# ===================== output folder =====================
out_dir = pathlib.Path(f"{root}/fibers_from_attn160_P97")
out_dir.mkdir(exist_ok=True)

# ===================== 遍历所有 attn160 文件 =====================
for npy_file in sorted(attn_folder.glob(f"*attn160.npy")):

    print("Processing:", npy_file.name)
    attn160 = np.load(npy_file)

    T = threshold_p97(attn160)
    fibers = extract_fibers(attn160, T)

    # 输出 txt 文件
    txt_path = out_dir / (npy_file.stem + "_fibers.txt")
    with open(txt_path, "w") as f:
        f.write(f"Threshold(P97): {T}\n")
        f.write("Fibers:\n")
        for name in fibers:
            f.write(f"  {name}\n")

print("🎉 Done! 所有 attn160.npy 的 fiber 名称已经全部输出")


In [ ]:

with torch.no_grad():   
    model.eval()
    preds = model(allimages)
mixed = [((labels==0)&(preds==labels)).float().sum(),
                 ((labels==0)&(preds!=labels)).float().sum(),
                 ((labels==1)&(preds==labels)).float().sum(),
                 ((labels==1)&(preds!=labels)).float().sum()]
prec = mixed[0]/(mixed[0]+mixed[1])
recall = mixed[0]/(mixed[0]+mixed[3])
f1 = 2*prec*recall/(prec+recall)

In [ ]:

with torch.no_grad():   
    model.eval()
    preds = model(allimages)
mixed = [((labels==0)&(preds==labels)).float().sum(),
                 ((labels==0)&(preds!=labels)).float().sum(),
                 ((labels==1)&(preds==labels)).float().sum(),
                 ((labels==1)&(preds!=labels)).float().sum()]
prec = mixed[0]/(mixed[0]+mixed[1])
recall = mixed[0]/(mixed[0]+mixed[3])
f1 = 2*prec*recall/(prec+recall)